In [1]:
import os

In [3]:
VERSION='22.05'
os.system(f"docker pull nvcr.io/nvidia/tritonserver:{VERSION}-py3")
# add --gpus=<number of gpus> on gpu machines
# add -d to run at background and going to the next cell
os.system("docker run --rm -d -p5000:8000 -p5001:8001 -p5002:8002"
          f" -v {os.getcwd()}/models:/models "
          f"nvcr.io/nvidia/tritonserver:{VERSION}-py3"
          " tritonserver --model-repository=/models")

22.05-py3: Pulling from nvidia/tritonserver
Digest: sha256:a85daa2907f46e70b3782818a0331df62d9b4e0b1f15f1530b2a52c8c782d46d
Status: Image is up to date for nvcr.io/nvidia/tritonserver:22.05-py3
nvcr.io/nvidia/tritonserver:22.05-py3
a1e048e71e092c5aa59609edb553b8a5ed92a3f2d2031929a8034d5e5f7f8afc


0

In [4]:
text = "Replace me by any text you'd like."
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer_dis = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer_dis

PreTrainedTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
encoded_input = tokenizer_dis(text, return_tensors='pt')
encoded_input.numpy()

AttributeError: 

In [12]:
import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException

try:
    triton_client = httpclient.InferenceServerClient(
        url='localhost:5000', verbose=True
    )
except Exception as e:
    print("context creation failed: " + str(e))

model_name = "distiluncase"
inputs = []
inputs.append(
    httpclient.InferInput(
        name="input", shape=encoded_input.shape, datatype="INT64")
)
inputs[0].set_data_from_numpy(encoded_input, binary_data=False)
 
outputs = []
outputs.append(httpclient.InferRequestedOutput(name="output"))
 
result = triton_client.infer(
    model_name=model_name, inputs=encoded_input, outputs=outputs)
triton_client.close()

AttributeError: 

In [18]:
# use onnx model
import onnx
import onnxruntime
model_dir = "models/distiluncase/1" 
model_path = os.path.join(model_dir, "model.onnx")
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)
 
ort_session = onnxruntime.InferenceSession(
    os.path.join(model_dir, "model.onnx"),
    providers=['CPUExecutionProvider'])


In [ ]:
onnx_model